## **Dados basicos como o token do telegram necessario e o nome da planilha**


In [ ]:
TOKEN_TELEGRAM = input('Digite o token do telegram:')
NOME_PLANILHA = input('Digite o nome da planilha onde os dados será salvo:')

## **Instalação da biblioteca necessaria**

In [ ]:
%pip install pytelegrambotapi

## **Autenticação com o google**

In [3]:
# Autenticação com o google

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
# Inportações de pacotes
import telebot
import pandas as pd
from datetime import datetime

In [5]:
spreedsheets = gc.open(NOME_PLANILHA)
mensagem_target2 = 'Resultado'

page = spreedsheets.sheet1

df = pd.DataFrame(page.get_all_values())

In [6]:
# Funções comuns
def salvarMensagem(text):
  date_now = datetime.today().strftime('%d-%m-%Y')

  if 'Green' in text or 'Red' in text:
    data = text.split('\n')

    if "tipmanager.com.br" in data:
      data.remove("tipmanager.com.br")

    if "🤖Bot criado por " in data:
      data.remove("🤖Bot criado por ")
      #data.remove("conosco e crie seus próprios bots!")

    newdata = [data[0], data[2], data[3], data[5], data[7], data[13], data[17], data[18]]

    horas_inicio = newdata[3].replace('🕒 Início: ', '')

    infor = newdata[0].split(' ')

    liga = f'{infor[0]} {infor[1]}'
    linha = infor[2]
    odd = infor[3].replace('@', '')
    confronto = newdata[1].replace('⚽   ', '') + ' ' + newdata[2]
    data_hora = f'{date_now} {horas_inicio}'
    estrategia = newdata[5].replace('❗ Estratégia: ', '')
    resultado = newdata[6].replace('Resultado: ', '')
    status = newdata[7].replace('✅ ', '').replace('❌ ', '')

    lucro = 0

    if status == 'Green':
      lucro = float(odd) - 1
    elif status == 'Red':
      lucro = -1
    elif status == 'Anulada':
      lucro = 0
    elif status == 'Meio Green':
      lucro = (float(odd) - 1)/2
    elif status == 'Meio Red':
      lucro = -0.5
    else:
      lucro = 0

    dicionario = [liga, confronto, data_hora, linha, odd,estrategia, resultado, lucro, status]

    page.append_rows([dicionario])

    return "Salvo com sucesso."

In [7]:
# Bot

bot = telebot.TeleBot(TOKEN_TELEGRAM)

try:
  @bot.edited_message_handler()
  def mensagemUpdade(message) -> None:
    if mensagem_target2 in message.text:
        save = salvarMensagem(message.text)
        bot.reply_to(message, str(save))

  @bot.message_handler()
  def mensagem(message) -> None:
    if mensagem_target2 in message.text:
        save = salvarMensagem(message.text)
        bot.reply_to(message, str(save))

except Exception as ex:
  print(ex)

bot.polling()